In [1]:
from typing import TypedDict, Literal, List
from langgraph.graph import StateGraph, START, END
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
import random


In [5]:

# 1단계: 상태 정의
class RecommendationState(TypedDict):
    customer_preference: str        # 고객 취향
    recommended_menu: str           # 추천된 메뉴명
    menu_details: str              # 메뉴 상세 정보
    recommendation_reason: str      # 추천 이유

# Vector DB 초기화
embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
menu_db = FAISS.load_local(
    "./db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)


In [6]:

# 2단계: 노드 함수들 정의

def get_customer_preference(state: RecommendationState) -> RecommendationState:
    """고객 취향을 파악하는 단계"""
    print("---고객 취향 파악---")
    preferences = ["단맛", "쓴맛", "신맛"]
    preference = random.choice(preferences)
    print(f"고객 취향: {preference}")
    
    return {"customer_preference": preference}


In [7]:

def recommend_sweet_menu(state: RecommendationState) -> RecommendationState:
    """단맛을 선호하는 고객을 위한 메뉴 추천 (Vector DB 검색)"""
    print("---단맛 메뉴 추천---")
    
    # 단맛 관련 키워드로 검색
    sweet_queries = ["바닐라", "카라멜", "달콤한", "티라미수"]
    query = random.choice(sweet_queries)
    
    # Vector DB에서 검색
    docs = menu_db.similarity_search(query, k=2)
    
    if docs:
        selected_doc = random.choice(docs)
        menu_name = selected_doc.metadata.get('menu_name', 'Unknown')
        menu_content = selected_doc.page_content
        
        reason = f"{menu_name}은(는) 달콤한 맛을 좋아하시는 고객님께 완벽한 선택입니다. Vector DB 검색을 통해 '{query}' 키워드와 가장 유사한 메뉴로 선정되었습니다."
        
        print(f"추천 메뉴: {menu_name}")
        print(f"추천 이유: {reason}")
        
        return {
            "recommended_menu": menu_name,
            "menu_details": menu_content,
            "recommendation_reason": reason
        }
    
    # 기본값
    return {
        "recommended_menu": "바닐라 라떼",
        "menu_details": "달콤한 바닐라 시럽이 들어간 라떼",
        "recommendation_reason": "달콤한 맛을 선호하는 고객님께 추천드립니다."
    }


In [8]:

def recommend_bitter_menu(state: RecommendationState) -> RecommendationState:
    """쓴맛을 선호하는 고객을 위한 메뉴 추천 (Vector DB 검색)"""
    print("---쓴맛 메뉴 추천---")
    
    # 쓴맛 관련 키워드로 검색
    bitter_queries = ["아메리카노", "콜드브루", "에스프레소", "원두"]
    query = random.choice(bitter_queries)
    
    # Vector DB에서 검색
    docs = menu_db.similarity_search(query, k=2)
    
    if docs:
        selected_doc = random.choice(docs)
        menu_name = selected_doc.metadata.get('menu_name', 'Unknown')
        menu_content = selected_doc.page_content
        
        reason = f"{menu_name}은(는) 진한 커피 맛을 좋아하시는 고객님께 완벽한 선택입니다. Vector DB 검색을 통해 '{query}' 키워드와 가장 유사한 메뉴로 선정되었습니다."
        
        print(f"추천 메뉴: {menu_name}")
        print(f"추천 이유: {reason}")
        
        return {
            "recommended_menu": menu_name,
            "menu_details": menu_content,
            "recommendation_reason": reason
        }
    
    # 기본값
    return {
        "recommended_menu": "아메리카노",
        "menu_details": "진한 에스프레소에 뜨거운 물을 더한 클래식 커피",
        "recommendation_reason": "진한 커피 맛을 선호하는 고객님께 추천드립니다."
    }


In [9]:

def recommend_sour_menu(state: RecommendationState) -> RecommendationState:
    """신맛을 선호하는 고객을 위한 메뉴 추천 (Vector DB 검색)"""
    print("---신맛 메뉴 추천---")
    
    # 신맛 관련 키워드로 검색
    sour_queries = ["녹차", "과일", "상큼한", "프라푸치노"]
    query = random.choice(sour_queries)
    
    # Vector DB에서 검색
    docs = menu_db.similarity_search(query, k=2)
    
    if docs:
        selected_doc = random.choice(docs)
        menu_name = selected_doc.metadata.get('menu_name', 'Unknown')
        menu_content = selected_doc.page_content
        
        reason = f"{menu_name}은(는) 상큼한 맛을 좋아하시는 고객님께 완벽한 선택입니다. Vector DB 검색을 통해 '{query}' 키워드와 가장 유사한 메뉴로 선정되었습니다."
        
        print(f"추천 메뉴: {menu_name}")
        print(f"추천 이유: {reason}")
        
        return {
            "recommended_menu": menu_name,
            "menu_details": menu_content,
            "recommendation_reason": reason
        }
    
    # 기본값
    return {
        "recommended_menu": "녹차 라떼",
        "menu_details": "말차 파우더와 스팀 밀크로 만든 건강한 음료",
        "recommendation_reason": "상큼한 맛을 선호하는 고객님께 추천드립니다."
    }


In [10]:

def decide_recommendation_path(state: RecommendationState) -> Literal["sweet", "bitter", "sour"]:
    """고객 취향에 따라 다음 실행할 노드를 결정하는 함수"""
    preference = state["customer_preference"]
    if preference == "단맛":
        return "sweet"
    elif preference == "쓴맛":
        return "bitter"
    else:
        return "sour"

# 3단계: 그래프 구성
builder = StateGraph(RecommendationState)

builder.add_node("get_preference", get_customer_preference)
builder.add_node("recommend_sweet", recommend_sweet_menu)
builder.add_node("recommend_bitter", recommend_bitter_menu)
builder.add_node("recommend_sour", recommend_sour_menu)

builder.add_edge(START, "get_preference")
builder.add_conditional_edges(
    "get_preference",
    decide_recommendation_path,
    {
        "sweet": "recommend_sweet",
        "bitter": "recommend_bitter",
        "sour": "recommend_sour"
    }
)
builder.add_edge("recommend_sweet", END)
builder.add_edge("recommend_bitter", END)
builder.add_edge("recommend_sour", END)

graph = builder.compile()


In [ ]:

# 4단계: 실행 및 테스트
initial_state = {
    "customer_preference": "",
    "recommended_menu": "",
    "menu_details": "",
    "recommendation_reason": ""
}

result = graph.invoke(initial_state)
print("\n=== 최종 추천 결과 ===")
print(f"고객 취향: {result['customer_preference']}")
print(f"추천 메뉴: {result['recommended_menu']}")
print(f"메뉴 상세: {result['menu_details']}")
print(f"추천 이유: {result['recommendation_reason']}")

In [ ]:
def extract_menu_info(doc: Document) -> dict:
    """문서에서 메뉴 정보 추출"""
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    
    # 가격 추출
    price_match = re.search(r'₩([\d,]+)', content)
    price = price_match.group(0) if price_match else "가격 정보 없음"
    
    # 설명 추출
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    description = description_match.group(1).strip() if description_match else "설명 없음"
    
    return {
        "name": menu_name,
        "price": price,
        "description": description,
        "full_content": content
    }

In [ ]:

# 2단계: 노드 함수들 정의

def classify_inquiry(state: CafeInquiryState) -> CafeInquiryState:
    """사용자 문의를 분류하는 단계"""
    print("---문의 유형 분류---")
    
    last_message = state["messages"][-1].content
    
    # 키워드 기반 분류
    if any(keyword in last_message for keyword in ["가격", "얼마", "비용"]):
        inquiry_type = "가격 문의"
    elif any(keyword in last_message for keyword in ["추천", "뭐가 좋", "어떤 게"]):
        inquiry_type = "추천 요청"
    else:
        inquiry_type = "메뉴 문의"
    
    print(f"분류된 문의 유형: {inquiry_type}")
    
    return {"inquiry_type": inquiry_type}

In [ ]:
def handle_price_inquiry(state: CafeInquiryState) -> CafeInquiryState:
    """가격 문의에 대한 응답 생성 (Vector DB 검색)"""
    print("---가격 문의 처리 (Vector DB 검색)---")
    
    # 전체 메뉴 검색을 위한 일반적인 쿼리
    docs = menu_db.similarity_search("메뉴 가격", k=5)
    
    response = "저희 카페의 메뉴별 가격은 다음과 같습니다:\n\n"
    
    if docs:
        for doc in docs:
            menu_info = extract_menu_info(doc)
            response += f"• {menu_info['name']}: {menu_info['price']}\n"
    else:
        response += "죄송합니다. 현재 가격 정보를 불러올 수 없습니다."
    
    response += "\n혹시 특정 메뉴에 대해 더 자세한 정보가 필요하시면 언제든 말씀해 주세요!"
    
    # 검색된 메뉴 정보 저장
    retrieved_menus = [extract_menu_info(doc)["name"] for doc in docs] if docs else []
    
    return {
        "messages": [AIMessage(content=response)],
        "retrieved_menus": retrieved_menus
    }

In [ ]:
def handle_recommendation(state: CafeInquiryState) -> CafeInquiryState:
    """추천 요청에 대한 응답 생성 (Vector DB 검색)"""
    print("---추천 요청 처리 (Vector DB 검색)---")
    
    # 사용자 메시지에서 키워드 추출하여 더 정확한 검색
    user_message = state["messages"][-1].content
    
    # 추천 관련 키워드로 검색
    docs = menu_db.similarity_search(user_message, k=3)
    
    if not docs:
        # 기본 검색
        docs = menu_db.similarity_search("인기 메뉴", k=3)
    
    response = "고객님께 추천드리는 메뉴는 다음과 같습니다:\n\n"
    
    if docs:
        # 랜덤으로 2개 선택
        selected_docs = random.sample(docs, min(2, len(docs)))
        
        for doc in selected_docs:
            menu_info = extract_menu_info(doc)
            response += f"🔸 {menu_info['name']} ({menu_info['price']})\n"
            response += f"   {menu_info['description'][:100]}...\n\n"
        
        response += "이 메뉴들은 Vector DB 검색을 통해 고객님의 문의와 가장 관련성 높은 메뉴로 선정되었습니다!"
    else:
        response += "죄송합니다. 현재 추천할 수 있는 메뉴 정보를 불러올 수 없습니다."
    
    # 검색된 메뉴 정보 저장
    retrieved_menus = [extract_menu_info(doc)["name"] for doc in docs] if docs else []
    
    return {
        "messages": [AIMessage(content=response)],
        "retrieved_menus": retrieved_menus
    }

In [ ]:
def handle_menu_inquiry(state: CafeInquiryState) -> CafeInquiryState:
    """일반 메뉴 문의에 대한 응답 생성 (Vector DB 검색)"""
    print("---메뉴 문의 처리 (Vector DB 검색)---")
    
    # 사용자 메시지를 바탕으로 검색
    user_message = state["messages"][-1].content
    docs = menu_db.similarity_search(user_message, k=4)
    
    if not docs:
        # 기본 검색
        docs = menu_db.similarity_search("카페 메뉴", k=4)
    
    response = "저희 카페에서는 다음과 같은 메뉴를 제공하고 있습니다:\n\n"
    
    if docs:
        for doc in docs:
            menu_info = extract_menu_info(doc)
            response += f"• {menu_info['name']}: {menu_info['description'][:80]}... ({menu_info['price']})\n\n"
        
        response += "위 메뉴들은 Vector DB 검색을 통해 고객님의 문의와 관련성이 높은 메뉴들입니다."
    else:
        response += "죄송합니다. 현재 메뉴 정보를 불러올 수 없습니다."
    
    response += "\n\n각 메뉴에 대한 자세한 정보나 추천이 필요하시면 언제든 말씀해 주세요!"
    
    # 검색된 메뉴 정보 저장
    retrieved_menus = [extract_menu_info(doc)["name"] for doc in docs] if docs else []
    
    return {
        "messages": [AIMessage(content=response)],
        "retrieved_menus": retrieved_menus
    }


In [ ]:
def decide_response_type(state: CafeInquiryState) -> Literal["price", "recommendation", "menu"]:
    """문의 유형에 따라 다음 실행할 노드를 결정"""
    inquiry_type = state["inquiry_type"]
    if inquiry_type == "가격 문의":
        return "price"
    elif inquiry_type == "추천 요청":
        return "recommendation"
    else:
        return "menu"

# 3단계: 그래프 구성
builder = StateGraph(CafeInquiryState)

builder.add_node("classify", classify_inquiry)
builder.add_node("handle_price", handle_price_inquiry)
builder.add_node("handle_recommendation", handle_recommendation)
builder.add_node("handle_menu", handle_menu_inquiry)

builder.add_edge(START, "classify")
builder.add_conditional_edges(
    "classify",
    decide_response_type,
    {
        "price": "handle_price",
        "recommendation": "handle_recommendation",
        "menu": "handle_menu"
    }
)
builder.add_edge("handle_price", END)
builder.add_edge("handle_recommendation", END)
builder.add_edge("handle_menu", END)

graph = builder.compile()

In [ ]:
# 4단계: 실행 및 테스트
def test_inquiry_system_with_vectordb():
    """Vector DB 연동 문의 시스템 테스트 함수"""
    test_messages = [
        "메뉴 가격이 궁금해요",
        "커피 중에서 어떤 메뉴를 추천해 주시나요?",
        "디저트 메뉴가 있나요?"
    ]

    for message in test_messages:
        print(f"\n{'='*60}")
        print(f"고객 문의: {message}")
        print('='*60)
        
        initial_state = {
            "messages": [HumanMessage(content=message)],
            "inquiry_type": "",
            "retrieved_menus": []
        }
        
        result = graph.invoke(initial_state)
        
        print(f"\n카페 응답:\n{result['messages'][-1].content}")
        print(f"\n검색된 메뉴들: {result['retrieved_menus']}")

# 테스트 실행
test_inquiry_system_with_vectordb()